# 🫁 Pneumothorax AI — Global Pre-training
**TÜBİTAK 2209-A | Ahmet Demir | Dokuz Eylül Üniversitesi**

Bu notebook:
1. GitHub'dan projeyi klonlar
2. Kaggle NIH ChestX-ray14 veri setini indirir
3. U-Net++ modelini 50 epoch eğitir
4. En iyi checkpoint'i Google Drive'a kaydeder

**Tahmini süre:** T4 GPU ile ~12 saat (50 epoch, 112k görüntü)

## 0. GPU Kontrolü

In [ ]:
import torch
print('GPU:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'YOK — Runtime > GPU seç!')
print('CUDA:', torch.version.cuda)
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

## 1. Projeyi Klonla

In [ ]:
import os

REPO = 'https://github.com/ahmetai-cell/pneumothorax-ai-detection'
PROJECT_DIR = '/content/pneumothorax-ai-detection'

if os.path.exists(PROJECT_DIR):
    !cd {PROJECT_DIR} && git pull
else:
    !git clone {REPO}

os.chdir(PROJECT_DIR)
print('Çalışma dizini:', os.getcwd())

## 2. Bağımlılıkları Yükle

In [ ]:
!pip install -q \
    segmentation-models-pytorch \
    albumentations \
    pydicom \
    pynrrd \
    wandb \
    tqdm \
    fpdf2 \
    plotly

print('✓ Kurulum tamamlandı')

## 3. Kaggle API Token Ayarla

In [ ]:
import os

# Kaggle token'ını buraya yapıştır:
KAGGLE_TOKEN = 'KGAT_ab6d3f32951d02e0cc877e406e207b72'  # salihekmen hesabı
KAGGLE_USER  = 'salihekmen'

os.environ['KAGGLE_API_TOKEN'] = KAGGLE_TOKEN
os.environ['KAGGLE_USERNAME']  = KAGGLE_USER

# kaggle.json oluştur (bazı araçlar için gerekli)
import json
kaggle_dir = os.path.expanduser('~/.kaggle')
os.makedirs(kaggle_dir, exist_ok=True)
with open(f'{kaggle_dir}/kaggle.json', 'w') as f:
    json.dump({'username': KAGGLE_USER, 'key': KAGGLE_TOKEN}, f)
os.chmod(f'{kaggle_dir}/kaggle.json', 0o600)

print('✓ Kaggle token ayarlandı')

## 4. NIH ChestX-ray14 İndir

In [ ]:
NIH_DIR = 'data/raw/global/nih'
os.makedirs(NIH_DIR, exist_ok=True)

# Zaten indirilmişse atla
nih_images = !find {NIH_DIR} -name '*.png' 2>/dev/null | wc -l
n_images = int(nih_images[0].strip())

if n_images > 100000:
    print(f'✓ NIH zaten mevcut: {n_images:,} görüntü')
else:
    print(f'NIH indiriliyor (~42 GB)...')
    !kaggle datasets download \
        -d nih-chest-xrays/data \
        -p {NIH_DIR} \
        --unzip
    nih_images = !find {NIH_DIR} -name '*.png' | wc -l
    print(f'✓ İndirme tamamlandı: {nih_images[0].strip()} görüntü')

## 5. Manifest Oluştur

In [ ]:
!python scripts/data_manager.py --build_manifest
!python scripts/unify_annotations.py

## 6. W&B Giriş

In [ ]:
import wandb

# wandb.ai/ahmet-ai-t-bi-tak hesabınla giriş yap
wandb.login()

## 7. Google Drive Bağla (Checkpoint Kaydet)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_CKPT = '/content/drive/MyDrive/tubitak_pneumothorax/checkpoints'
os.makedirs(DRIVE_CKPT, exist_ok=True)
print(f'✓ Drive bağlandı: {DRIVE_CKPT}')

## 8. Pre-training Başlat

**T4 GPU'da tahmini süre:**
- 1 epoch (112k görüntü, batch=32): ~14 dakika
- 50 epoch: ~12 saat

> ⚠️ Colab oturumu ~12 saatte kapanabilir. Checkpoint her fold'dan sonra kaydedilir.

In [ ]:
!python scripts/train_global.py \
    --sources NIH \
    --encoder efficientnet-b0 \
    --img_size 512 \
    --epochs 50 \
    --batch_size 32 \
    --num_folds 5 \
    --lr 1e-4

## 9. Checkpoint'i Drive'a Kopyala

In [ ]:
import shutil, glob

# Base model
base_model = 'checkpoints/global_base_model.pth'
if os.path.exists(base_model):
    shutil.copy(base_model, f'{DRIVE_CKPT}/global_base_model.pth')
    shutil.copy('checkpoints/global_base_model_meta.json',
                f'{DRIVE_CKPT}/global_base_model_meta.json')
    print(f'✓ Base model Drive\'a kopyalandı')

# Tüm fold checkpoint'leri
for ckpt in glob.glob('checkpoints/global_folds/*.pth'):
    shutil.copy(ckpt, DRIVE_CKPT)
    print(f'  → {os.path.basename(ckpt)}')

# Results CSV
results_csv = 'results/global_kfold_results.csv'
if os.path.exists(results_csv):
    shutil.copy(results_csv, DRIVE_CKPT)
    print(f'✓ Results CSV kopyalandı')

print('\n✓ Tüm dosyalar Drive\'a kaydedildi:', DRIVE_CKPT)

## 10. Sonuçlar

In [ ]:
import pandas as pd
import json

# K-Fold sonuçları
try:
    df = pd.read_csv('results/global_kfold_results.csv')
    print('=== K-FOLD SONUÇLARI ===')
    print(df.to_string(index=False))
    print(f'\nOrtalama Dice : {df["best_dice"].mean():.4f} ± {df["best_dice"].std():.4f}')
    print(f'Ortalama AUC  : {df["best_auc"].mean():.4f} ± {df["best_auc"].std():.4f}')
except FileNotFoundError:
    print('Results CSV bulunamadı — eğitim henüz bitmemiş olabilir')

# Base model meta
try:
    meta = json.load(open('checkpoints/global_base_model_meta.json'))
    print('\n=== BASE MODEL ===')
    for k, v in meta.items():
        print(f'  {k}: {v}')
except FileNotFoundError:
    pass

---
## ✅ Sonraki Adım — Fine-tuning

DEU DICOM + NRRD verileri geldiğinde:

```bash
# Checkpoint'i Drive'dan al
cp /content/drive/MyDrive/tubitak_pneumothorax/checkpoints/global_base_model.pth checkpoints/

# DEU verilerini koy
# data/local/dicom/*.dcm
# data/local/nrrd/*.nrrd

# Fine-tune et (~1-2 saat)
python scripts/fine_tune_local.py --freeze_encoder --epochs 20
```